# WID-U Compute
This notebooks processes image crops with a trained u-net
## Usage
Click on the play buttons below and follow the instructions.
1. Upload the trained u-net
2. Upload the zip file containing the images to process
2. Download the zip files containing the results

In [ ]:
## Click the play button, then upload the trained u-net (hdf5 or h5 format)
from google.colab import files
uploadedModel = files.upload()

Saving cover.PNG to cover.PNG


In [ ]:
## Click the play button, then upload the ZIP file with the exported images to process
from google.colab import files
uploadedImages = files.upload()

User uploaded file "cover.PNG" with length 1770296 bytes


In [ ]:
################################################################################
# wid-u rev 1.2                                                                #
# converts a transmitted light image sequence to pseudofluorescence            #
################################################################################

# SETTINGS #####################################################################
im_width = 224
im_height = 224
PATH_CROPS = '/content/images/'
PATH_SAVE = '/content/results/'

# DEPENDENCIES #################################################################
from google.colab import files
import os
import glob
from tqdm import tqdm_notebook

import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.preprocessing.image import img_to_array, load_img

import skimage.io as io
import skimage.transform as trans
from skimage import img_as_ubyte

# UTILS ########################################################################
Class1 = [255,255,255]
Unlabelled = [0,0,0]
COLOR_DICT = np.array([Unlabelled, Class1])

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResults(save_path,npyfile,ids,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        fn = ids[i]
        io.imsave(os.path.join(save_path,fn+'.png'),img_as_ubyte(img))

# MAIN #########################################################################
## Load trained model
fn_model = ''
for fn in uploadedModel.keys():
  fn_model = fn
  print('User uploaded MODEL "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploadedModel[fn])))
print('Loading model ... ')
model = keras.models.load_model(fn_model)
X = np.zeros((1, im_height, im_width, 1), dtype=np.float32)

## Load images
fn_images = ''
for fn in uploadedImages.keys():
  fn_images = fn
  print('User uploaded IMAGES "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploadedImages[fn])))

!unzip fn_images -d PATH_CROPS

## PROCESS IMAGES
print('Processing images ... ')
for file in glob.glob(PATH_CROPS+"/*.png"):
    fn = os.path.basename(file)
    print(fn)
    ids = []
    ids.append(fn)
    img = load_img(path + '/' + fn, grayscale=True)
    x_img = img_to_array(img)
    X[0, ..., 0] = x_img.squeeze() / 255
    results = model.predict(X, verbose=1)
    saveResults(PATH_SAVE,results,ids)

## Zip results
!cp PATH_CROPS+'/coords.mat' PATH_SAVE+'/coords.mat'
!zip -r 'WIDU-PROCESSED.zip' PATH_SAVE+'*.*'

ImportError: ignored